<a href="https://colab.research.google.com/github/Maxindrull2/UFV_Visualicaci-n-de-datos/blob/main/clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install plotly_express

Se carga la base de datos

In [45]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('datos_ejercicio_ventas.csv')

df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


COUNTRY: Pais

SUBBRAND: Producto

YEAR: Año

MONTH: Mes

SCENARIO:
* Actual --> Real
* Forecast --> Predicción

FORECAST: Mes de la predicción + 1 (PF = P13F)

FORECAST_YEAR: Año en que se hizo la predicción

AMOUNT: Cantidad

# **Ejercicio 0**

1. Nº de actuals y forecast
2. Horizonte de previsión --> Solamente hace falta 1 pais --> Nº de puntos de predicciones
3. Nº de paises, productos...
4. Historico de datos de actuals/forecast
5. Forecast distintos
6. Rangos de amount

# **Ejercicio 1 - Distribución de ventas**

## Limpieza

In [46]:
print(df['SCENARIO'].value_counts())

SCENARIO
AI_forecast    17766
actual           900
Name: count, dtype: int64


Se cogen unicamente las ventas y no las predicciones

In [47]:
# prompt: del dataframe df coge unicamente aquellas columnas con SCENARIO=actual

df_ventas = df[df['SCENARIO'] == 'actual']

df_ventas.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
277,Portugal,Pepsi Max (L3),2023,10,actual,NaN,NaN,1.885949e+05
278,Portugal,7up (L3),2023,3,actual,NaN,NaN,2.934971e+05
279,Portugal,7up (L3),2023,10,actual,NaN,NaN,3.484466e+05
280,Great Britain,7up Free (L3),2023,10,actual,NaN,NaN,1.172553e+06
281,Norway,Pepsi Regular (L3),2023,10,actual,NaN,NaN,3.784859e+04


Los datos de forecast y forecast year deberian de ser NaN

In [48]:
print(df_ventas['SCENARIO'].unique())
print(df_ventas['FORECAST'].unique())
print(df_ventas['FORECAST_YEAR'].unique())

['actual']
[nan]
[nan]


Las columnas scenario, forecast y forecast year ya no son necesarias debido a que todos los datos son iguales

In [49]:
df_ventas = df_ventas.drop(['SCENARIO' ,'FORECAST', 'FORECAST_YEAR'], axis=1)

df_ventas.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,AMOUNT
277,Portugal,Pepsi Max (L3),2023,10,1.885949e+05
278,Portugal,7up (L3),2023,3,2.934971e+05
279,Portugal,7up (L3),2023,10,3.484466e+05
280,Great Britain,7up Free (L3),2023,10,1.172553e+06
281,Norway,Pepsi Regular (L3),2023,10,3.784859e+04


In [50]:
print(df_ventas['COUNTRY'].unique())
print("Numero de paises únicos:", len(df_ventas['COUNTRY'].unique()))
print('\n',df_ventas['SUBBRAND'].unique())
print("Numero de productos únicos:", len(df_ventas['SUBBRAND'].unique()))
print('\n',df_ventas['YEAR'].unique())
print("Numero de años únicos:", len(df_ventas['YEAR'].unique()))
print('\n',df_ventas['MONTH'].unique())
print("Numero de meses únicos:", len(df_ventas['MONTH'].unique()))

['Portugal' 'Great Britain' 'Norway' 'Netherlands' 'Hungary' 'Spain'
 'Denmark' 'Czech' 'Italy']
Numero de paises únicos: 9

 ['Pepsi Max (L3)' '7up (L3)' '7up Free (L3)' 'Pepsi Regular (L3)'
 'Lipton (L3)' 'Mountain Dew (L3)']
Numero de productos únicos: 6

 [2023 2024]
Numero de años únicos: 2

 [10  3  1  2  4  7  5 11 12  6  8  9]
Numero de meses únicos: 12


Confirmamos que no existan fechas incorrectas (que aun no hayan sucedido)

In [51]:
print(sorted(df_ventas[df_ventas['YEAR'] == 2024]['MONTH'].unique()))

[1, 2, 3, 4, 5, 6, 7, 8]


Se ve la estructura del amount

In [52]:
print(df_ventas['AMOUNT'].describe())

count    9.000000e+02
mean     7.443485e+05
std      1.905511e+06
min     -2.171201e+05
25%      5.574319e+04
50%      1.856788e+05
75%      6.432082e+05
max      1.481563e+07
Name: AMOUNT, dtype: float64


### Gráficos exploratorios

In [62]:
import plotly.graph_objects as go

fig = go.Figure()

for country in df_ventas['COUNTRY'].unique():
  df_country = df_ventas[df_ventas['COUNTRY'] == country]
  df_country = df_country.drop('SUBBRAND', axis=1)
  df_country = df_country.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()
  df_country['Date'] = pd.to_datetime(df_country['YEAR'].astype(str) + '-' + df_country['MONTH'].astype(str), format='%Y-%m')
  df_country = df_country.sort_values('Date')
  fig.add_trace(go.Scatter(x=df_country['Date'], y=df_country['AMOUNT'], mode='lines', name=country))

fig.update_layout(title='Ventas en cada país a lo largo del tiempo',
                  xaxis_title='Fecha',
                  yaxis_title='Cantidad')
fig.show()